<a href="https://colab.research.google.com/github/Blackman9t/Coursera_Capstone/blob/master/applied_data_science_capstone_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Applied Data Science Capstone-Project


### This Capsone-Project is the final course in a series of nine(9) Data Science Specialization courses taught by IBM via Coursera. <br>
###The courses include the following:-
1. [What is Data Science?](https://www.coursera.org/learn/what-is-datascience/home/welcome)
2. [Open Source tools for Data Science](https://www.coursera.org/learn/open-source-tools-for-data-science/home/welcome)
3. [Python for Data Science](https://www.coursera.org/learn/python-for-applied-data-science/home/welcome)
4. [Data Science Methodology](https://www.coursera.org/learn/data-science-methodology/home/welcome)
5. [Databases and SQL for Data Science](https://www.coursera.org/learn/sql-data-science/home/welcome)
6. [Data Visualization with Python](https://www.coursera.org/learn/python-for-data-visualization/home/welcome)
7. [Data Analysis with Python](https://www.coursera.org/learn/data-analysis-with-python/home/welcome)
8. [Machine Learning with Python](https://www.coursera.org/learn/machine-learning-with-python/home/welcome)
9. [Applied Data Science Capstone](https://www.coursera.org/learn/applied-data-science-capstone/home/welcome)

#### It is mandatory that all courses from 1 to 8 are passed with acceptable grades before attempting this Capsone-Project.

Importing requred libraries

In [0]:
import pandas as pd
import numpy as np

In [160]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


<h2>Problem Description</h2>

<h3>The problem I want to solve using Geo-location Data is helping a new Tech-StartUp find the ideal location for its Head-office in The City of Lagos Nigeria.
This Head-office should be situated in a good location that has the following:-</h3><br>

central location.<br>
Close proximity to Tech-Hubs and talent.<br>
High feet traffic area for easy interaction with potential customers.<br>
Nearness to Educational institutions for research and interactions with academics.<br>
Nearness to cafes and restaurants for business meetings or lunch-meets.<br>
Nearness to bus-stops, seaports and airports.<br>
Security and safety.<br>
Cluster of economic activities and similar or complementing businesses.


<h2>Background to Problem</h2>

I have lived in Lagos for some time off and on, longest has been 3 years stretch from 2006 to 2009. <br>
I have also done a lot of travelling to other Countries such as Russia, South-Africa, Ghana, Dubai, Mauritius and a few others.<br>
By the way, Mauritius is The wealthiest country in Africa with average wealth per person increasing from $(21000 - 25000) within one year recently ([link](https://www.graphic.com.gh/business/business-news/accra-ranked-10th-wealthiest-city-in-africa.html)).<br>
My analysis today is on Lagos state, which is the 4th wealthiest city in Africa, fourth place behind Johannesburg, Cairo and Cape Town ([link](https://www.graphic.com.gh/business/business-news/accra-ranked-10th-wealthiest-city-in-africa.html)).<br>
Lagos city is home to over 6,800 millionaires, 370 multi-millionaires and 4 billionaires.<br>
Lagos is also the commercial capital of Nigeria (The Most populous black nation), and the commercial hub of West Africa.<br>
I have just only recently returned to Lagos city, got enrolled in The Founder Institute and currently building my new Startup in The Fintech industry ([link](http://bit.ly/escrowallet)).<br>
Therefore I want to use this opportunity to help future Startups find the most ideal location in Lagos city, for a Head-office.

<h2>Data Requirements and Overview</h2>

**The Dataset is the wikipedia page of Lagos state** see [link](https://en.wikipedia.org/wiki/List_of_Lagos_State_local_government_areas_by_population)

We shall explore Lagos city through its respective Local Government Areas or Boroughs via this [link](https://en.wikipedia.org/wiki/List_of_Lagos_State_local_government_areas_by_population).<br>
The above link is a web page that shows the respective Local Government Areas in Lagos State and each population figure. It's a public wikipedia data page<br>
This data will be analysed through the following steps:-<br>
1. We shall scrape the web page using the beautiful soup library<br>
2. We shall use The Foresquare API calls to retrieve geo-location data<br>
3. We shall fetch the text data using the requests library<br>
4. We shall convert it from JSON to Pandas dataframe using the josn_normalize module<br>
5. We shall use The folium library to view the maps and plot these via matplotlib library.<br>
Then we shall cluster Lagos City into its respective Postcodes and Local Government Areas and analyse each area for its appeal as a location for a Tech-Statrup based on the aforementioned parameters.<br>
After this we shall select our Top-Location.<br>
5. Finally to add some fun, we shall use the Word-cloud library to display the names of the top hubs, restaurants, offices and education institutes in our Top-Location.

**So please sit back, relax and enjoy the ride as we explore Lagos City**

First let's import required libraries

In [161]:
from bs4 import BeautifulSoup
import requests  # library to handle requests
import pandas as pd
import json  # library to handle JSON files
from pandas.io.json import json_normalize  # transform json files to pandas dataframes
from geopy.geocoders import Nominatim  # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import numpy as np
import csv

print('All modules imported')

All modules imported


In [0]:
# Lets save the weblink for The Lagos Data

lagos_link = 'https://en.wikipedia.org/wiki/List_of_Lagos_State_local_government_areas_by_population'

Get the source code html data from the website

In [0]:
source = requests.get(lagos_link).text

Lets Use BeautifulSoup to parse it

In [0]:
soup = BeautifulSoup(source,'lxml')

# Let's pretty print it in its right shape

#print(soup.prettify())

Next let's get the table that contains the data we want to scrape

In [0]:
my_table = soup.find_all('td')
#mw-content-text
# Let's view the table

# my_table

Next let's iterate through each link and append the text parts

In [0]:
table_text = []

for data in my_table:
    table_text.append(data.text)

In [167]:
# Let's see the text

table_text

['',
 'This article needs to be updated. Please update this article to reflect recent events or newly available information. (March 2015)',
 '',
 'This article does not cite any sources. Please help improve this article by adding citations to reliable sources. Unsourced material may be challenged and removed. (June 2015) (Learn how and when to remove this template message)',
 '1\n',
 'Alimosho\n',
 '5,700,714\n',
 '2\n',
 'Ajeromi-Ifelodun\n',
 '1,746,634\n',
 '3\n',
 'Kosofe\n',
 '665,998\n',
 '4\n',
 'Mushin, Lagos\n',
 '633,543\n',
 '5\n',
 'Oshodi-Isolo\n',
 '621,789\n',
 '6\n',
 'Ojo\n',
 '598,332\n',
 '7\n',
 'Ikorodu\n',
 '535,811\n',
 '8\n',
 'Surulere\n',
 '504,409\n',
 '9\n',
 'Agege\n',
 '461,123\n',
 '10\n',
 'Ifako-Ijaiye\n',
 '428,812\n',
 '11\n',
 'Somolu\n',
 '402,992\n',
 '12\n',
 'Amuwo-Odofin\n',
 '318,576\n',
 '13\n',
 'Lagos Mainland\n',
 '317,980\n',
 '14\n',
 'Ikeja\n',
 '313,333\n',
 '15\n',
 'Eti-Osa\n',
 '287,958\n',
 '16\n',
 'Badagry\n',
 '241,437\n',
 '17\n

First let's attract only the relevent data from the table

In [168]:
relevant_table_data = table_text[4:-3]

# Next let's see the first 3 elements
relevant_table_data[:3]

['1\n', 'Alimosho\n', '5,700,714\n']

Next lets create a dictionary and append the LGA and corresponding Population data to it

In [0]:
table_dict={'LGA':[], 'POP':[]}
count = 0

for item in relevant_table_data:
    # First let's strip off the \n at the end
    item = item.strip('\n')
    try:
        item = int(item)
    except:
        # if second item after the int, append to POP
        if count > 0:
            # First let's remove the commas
            item = item.replace(',','')
            # Next let's convert to an integer so we can use it for calculations
            item = int(item)
            # Finally, let's append it to the Population list of the dictionary
            table_dict['POP'].append(item)
            count = 0
        else:
         # if first item after the int, append to LGA
            table_dict['LGA'].append(item)
            count +=1
            

Next let's create a Dataframe of each LGA and its respective Population

In [170]:
lagos_df = pd.DataFrame(table_dict)

# Let's see the first five rows
lagos_df.head()

,LGA,POP
0,Alimosho,5700714
1,Ajeromi-Ifelodun,1746634
2,Kosofe,665998
3,"Mushin, Lagos",633543
4,Oshodi-Isolo,621789


Appending the Latitude and Longitude Values:<br>
First let's define a simple method to extract each LGA or Borough Lats and Longs data

In [0]:
def latitude_longitude(Borough):
    """ Method takes a Series object and returns
    a list of Latitude and corresponding Longitude data,
    using the geopy library.
    This method also prints out the coordinate data"""
    
    address = Borough
    
    geolocator = Nominatim(user_agent="NG_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
    return [latitude, longitude]


In [172]:
lagos_df['latitude'] = lagos_df['LGA'].apply(latitude_longitude)

The geograpical coordinates of Alimosho are 6.6113371, 3.2952634.
The geograpical coordinates of Ajeromi-Ifelodun are 6.4551217, 3.33594625182987.
The geograpical coordinates of Kosofe are 6.581974, 3.41483594161784.
The geograpical coordinates of Mushin, Lagos are 6.53263245, 3.35202197179877.
The geograpical coordinates of Oshodi-Isolo are 6.54001, 3.31241461318394.
The geograpical coordinates of Ojo are 6.46249745, 3.16570641436411.
The geograpical coordinates of Ikorodu are 6.6191233, 3.5041271.
The geograpical coordinates of Surulere are 8.09558835, 4.41296180365728.
The geograpical coordinates of Agege are 6.6229165, 3.3203386.
The geograpical coordinates of Ifako-Ijaiye are 6.632608, 3.3407753.
The geograpical coordinates of Somolu are 6.583333, 3.466667.
The geograpical coordinates of Amuwo-Odofin are 6.4470234, 3.26627971168259.
The geograpical coordinates of Lagos Mainland are 6.49306105, 3.38824963720956.
The geograpical coordinates of Ikeja are 6.5960605, 3.340787.
The geog

Let's see the data frame again

In [173]:
lagos_df.head()

,LGA,POP,latitude
0,Alimosho,5700714,"[6.6113371, 3.2952634]"
1,Ajeromi-Ifelodun,1746634,"[6.4551217, 3.33594625182987]"
2,Kosofe,665998,"[6.581974, 3.41483594161784]"
3,"Mushin, Lagos",633543,"[6.53263245, 3.35202197179877]"
4,Oshodi-Isolo,621789,"[6.54001, 3.31241461318394]"


Next let's loop through the data frame and separate Latitude from Longitude and make lat and lons just numbers not lists


In [0]:
lon_list = []
for i, j in lagos_df.iterrows():
    lon_list.append(j.latitude[1])
    lagos_df.iat[i,2] = j.latitude[0]
      
# next let's assign the lon_list as the value of the Longitude Column

lagos_df['longitude'] = lon_list

Let's see our table with the separated latitude and longitude values

In [175]:
lagos_df.head()

,LGA,POP,latitude,longitude
0,Alimosho,5700714,6.61134,3.295263
1,Ajeromi-Ifelodun,1746634,6.45512,3.335946
2,Kosofe,665998,6.58197,3.414836
3,"Mushin, Lagos",633543,6.53263,3.352022
4,Oshodi-Isolo,621789,6.54001,3.312415


Okay so at this point let's look at each Borough or L.G.A closer. Let's start with the first one and visualize it via Folium library

Let's get the geographic coordinates of Alimosho, the first L.G.A

In [176]:
alimosho_data = lagos_df.loc[0,].to_frame().T
alimosho_data

,LGA,POP,latitude,longitude
0,Alimosho,5700714,6.61134,3.29526


In [177]:
lat, lon = latitude_longitude(alimosho_data.iloc[0,0])

print('lat: {}, lon: {}'.format(lat,lon))

The geograpical coordinates of Alimosho are 6.6113371, 3.2952634.
lat: 6.6113371, lon: 3.2952634


In [178]:
# create map of Alimosho using latitude and longitude values
map_alimosho = folium.Map(location=[lat, lon], zoom_start=14)

# add markers to map

label = "{}".format('Alimosho')
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
    [lat, lon],
    radius=10,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.8,
    parse_html=False).add_to(map_alimosho)  

map_alimosho  # Feel free to zoom in to see more

Now Let's explore the data using foursquare API

In [179]:
CLIENT_ID = 'L2XUYCRD44VP2YBDMZ3XY0Z4XKO1JODM5SMG3IIYRZ5330FD' # your Foursquare ID
CLIENT_SECRET = '5KI00FG4RT2VJQTIDPXFY1N4AOKTIXLIZEAO0AHPP3DPD0U1' # your Foursquare Secret
VERSION = '20190902'
LIMIT = 50
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: L2XUYCRD44VP2YBDMZ3XY0Z4XKO1JODM5SMG3IIYRZ5330FD
CLIENT_SECRET:5KI00FG4RT2VJQTIDPXFY1N4AOKTIXLIZEAO0AHPP3DPD0U1


Let's get the full data for Alimosho LGA

Now, let's get the top 200 venues that are in the first entry within a radius of 15 Kilometers.

In [0]:
radius = 10000 # 10,000 meters is 10KM
LIMIT=200

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    alimosho_data.latitude[0], 
    alimosho_data.longitude[0], 
    radius, 
    LIMIT)

let's Send the GET request and examine the resutls

In [181]:
result = requests.get(url).json()

result.keys()

dict_keys(['meta', 'response'])

In [0]:
# show the result
# result

Let's define a function that extracts the category of the venue

Let's save only the venues from result in a variable called venues

In [183]:
venues = result['response']['groups'][0]['items']
len(venues)

# This tells us 88 venues are returned in a 10 Kilometer radius of Alimosho LGA

86

Next, lets convert the result of nearby venues returned into a DataFrame

In [184]:
nearby_venues = json_normalize(venues)
nearby_venues.head()

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.neighborhood,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4e261fce315161d79d2a8fa5-0,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",4e261fce315161d79d2a8fa5,federal junction beside mr biggs gowon estate ...,NG,Lagos,Nigeria,NaN,2111,[federal junction beside mr biggs gowon estate...,"[{'label': 'display', 'lat': 6.604608729562348...",6.604609,3.277413,NaN,+234,Lagos,SHAWARMA.COME,0,[],NaN
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4f75a8e7e4b06c81e0e3ba05-1,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",4f75a8e7e4b06c81e0e3ba05,NaN,NG,Lagos,Nigeria,NaN,5168,"[Lagos, Lagos, Nigeria]","[{'label': 'display', 'lat': 6.576022675981682...",6.576023,3.325612,NaN,NaN,Lagos,Wings Restaurant,0,[],NaN
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ecbf689b6346e2d3a4dead5-2,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",4ecbf689b6346e2d3a4dead5,Abule Egba,NG,Lagos,Nigeria,NaN,5348,"[Abule Egba, Lagos, Lagos, Nigeria]","[{'label': 'display', 'lat': 6.649299147190656...",6.649299,3.265609,NaN,NaN,Lagos,Tantalisers,0,[],NaN
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4e8f081c77c843122cdaa1ad-3,"[{'id': '4bf58dd8d48988d104951735', 'name': 'B...",4e8f081c77c843122cdaa1ad,"117, Akowonjo Road",NG,Egbeda-Akowonjo,Nigeria,Jimoh Bus Stop,1289,"[117, Akowonjo Road (Jimoh Bus Stop), Egbeda-A...","[{'label': 'display', 'lat': 6.601810454245876...",6.601810,3.301896,NaN,NaN,Lagos,Pink Ice Boutique,0,[],NaN
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-5205f22911d2055531e5599b-4,"[{'id': '4f4528bc4b90abdf24c9de85', 'name': 'A...",5205f22911d2055531e5599b,NaN,NG,Lagos,Nigeria,NaN,2216,"[Lagos, Lagos, Nigeria]","[{'label': 'display', 'lat': 6.59356167915684,...",6.593562,3.304300,NaN,NaN,Lagos,"National Stadium, Surulere",0,[],NaN


Okay let's now filter the venue DataFrame and select only the volumns needed 

In [185]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.location.distance', 'venue.id']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng,venue.location.distance,venue.id
0,SHAWARMA.COME,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",6.604609,3.277413,2111,4e261fce315161d79d2a8fa5
1,Wings Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",6.576023,3.325612,5168,4f75a8e7e4b06c81e0e3ba05
2,Tantalisers,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",6.649299,3.265609,5348,4ecbf689b6346e2d3a4dead5
3,Pink Ice Boutique,"[{'id': '4bf58dd8d48988d104951735', 'name': 'B...",6.601810,3.301896,1289,4e8f081c77c843122cdaa1ad
4,"National Stadium, Surulere","[{'id': '4f4528bc4b90abdf24c9de85', 'name': 'A...",6.593562,3.304300,2216,5205f22911d2055531e5599b


We can see that the categories column is not well formatted. Let's clean it out.<br>
First let's inspect it closer

In [186]:
nearby_venues['venue.categories'][0]

[{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
   'suffix': '.png'},
  'id': '4bf58dd8d48988d1ca941735',
  'name': 'Pizza Place',
  'pluralName': 'Pizza Places',
  'primary': True,
  'shortName': 'Pizza'}]

Ok so let;s now define a simple method that returns only the name for each category.<br>
Then use the apply method to apply it on each category item in the dataframe
                                                            

In [0]:
def clean_category(x):
    x = x[0]['name']
    return x

Now let's apply it

In [0]:
nearby_venues['venue.categories'] = nearby_venues['venue.categories'].apply(clean_category)

Finally let's view the cleaned category column

In [189]:
alimosho_venues = nearby_venues

alimosho_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng,venue.location.distance,venue.id
0,SHAWARMA.COME,Pizza Place,6.604609,3.277413,2111,4e261fce315161d79d2a8fa5
1,Wings Restaurant,Restaurant,6.576023,3.325612,5168,4f75a8e7e4b06c81e0e3ba05
2,Tantalisers,Burger Joint,6.649299,3.265609,5348,4ecbf689b6346e2d3a4dead5
3,Pink Ice Boutique,Boutique,6.601810,3.301896,1289,4e8f081c77c843122cdaa1ad
4,"National Stadium, Surulere",Athletics & Sports,6.593562,3.304300,2216,5205f22911d2055531e5599b


Lets see the different types of venues present

In [190]:
alimosho_cat_count = alimosho_venues.groupby('venue.categories').size().to_frame()

# Let's see the categories of venues present in the alimosho LGA
alimosho_cat_count.index

Index(['African Restaurant', 'Airport', 'Airport Lounge',
       'American Restaurant', 'Athletics & Sports', 'BBQ Joint', 'Bakery',
       'Bar', 'Beer Garden', 'Boutique', 'Burger Joint', 'Café', 'Campground',
       'Chinese Restaurant', 'Clothing Store', 'Coffee Shop',
       'Convenience Store', 'Department Store', 'Electronics Store',
       'Farmers Market', 'Fast Food Restaurant', 'Fried Chicken Joint',
       'Heliport', 'Hotel', 'Ice Cream Shop', 'Lounge', 'Market', 'Multiplex',
       'Nightclub', 'Park', 'Performing Arts Venue', 'Pizza Place', 'Pool',
       'Restaurant', 'Seafood Restaurant', 'Shopping Mall', 'Soup Place',
       'Spa', 'Steakhouse', 'Wine Shop'],
      dtype='object', name='venue.categories')

<h3>Next let's encapsulate all these steps into a single method that we can call</h3>

First let's define a method for plotting each map

Let's create a copy of the original lagos_df so that we can make the LGA its index

In [191]:
# First let's make a copy of the lagos_df
copy_lagos_df = lagos_df.copy(deep=True)

# Then let's make LGAs the indes
copy_lagos_df.index = copy_lagos_df.LGA

# Let's drop the LGA Column
copy_lagos_df = copy_lagos_df.drop('LGA', axis=1)

# Lets view the effect
copy_lagos_df.head()

,POP,latitude,longitude
LGA,,,
Alimosho,5700714,6.61134,3.295263
Ajeromi-Ifelodun,1746634,6.45512,3.335946
Kosofe,665998,6.58197,3.414836
"Mushin, Lagos",633543,6.53263,3.352022
Oshodi-Isolo,621789,6.54001,3.312415


In [0]:
def plot_map(lga):
    """ This method takes an LGA name and
    returns a plotted map of it based on lat and lon"""
    lat = copy_lagos_df.loc[lga, 'latitude']
    lon = copy_lagos_df.loc[lga, 'longitude']
    
    # create map of Alimosho using latitude and longitude values
    map_lga = folium.Map(location=[lat, lon], zoom_start=14)

    # add markers to map

    label = "{}".format(lga)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.8,
        parse_html=False).add_to(map_lga)  
    return map_lga

Next let's define a method that returns a filtered out version of 200 popular venues within 10KM radius of the LGA

In [0]:
def return_venues(lga):
    
    # let's save the latitude and longitude values
    lat = copy_lagos_df.loc[lga, 'latitude']
    lon = copy_lagos_df.loc[lga, 'longitude']
    
    # First lets save the foursquare credentials
    CLIENT_ID = 'L2XUYCRD44VP2YBDMZ3XY0Z4XKO1JODM5SMG3IIYRZ5330FD' # your Foursquare ID
    CLIENT_SECRET = '5KI00FG4RT2VJQTIDPXFY1N4AOKTIXLIZEAO0AHPP3DPD0U1' # your Foursquare Secret
    VERSION = '20190902'
    LIMIT = 50
#     print('Your credentials:')
#     print('CLIENT_ID: ' + CLIENT_ID)
#     print('CLIENT_SECRET:' + CLIENT_SECRET)
    
    # Now, let's get the top 200 venues that are in the first entry within a radius of 15 Kilometers.
    radius = 10000 # 10,000 meters is 10KM
    LIMIT=200  # no mire than 200 venues
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lon, 
        radius, 
        LIMIT)
    
    # Let's filter out the result
    result = requests.get(url).json()
    
    # From the result, lets get the venues
    venues = result['response']['groups'][0]['items']
    
    # Lets convert the json files to dataframe
    nearby_venues = json_normalize(venues)
    
    # Let's filter the venues and select only columns needed
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng', 'venue.location.distance', 'venue.id']
    nearby_venues = nearby_venues.loc[:, filtered_columns]
    
    # Next lets use the clean_category method to clean out category column
    nearby_venues['venue.categories'] = nearby_venues['venue.categories'].apply(clean_category)

    
    return nearby_venues

    


Let's see the number of venues returned for the second LGA and the first 5 venues

In [194]:
# first let's confirm the name
copy_lagos_df.index[1]

'Ajeromi-Ifelodun'

In [195]:
Ajeromi_lga_venues = return_venues(copy_lagos_df.index[1])

print('Total number of venues returned is {}'.format(len(Ajeromi_lga_venues)))

Ajeromi_lga_venues.head()


Total number of venues returned is 61


,venue.name,venue.categories,venue.location.lat,venue.location.lng,venue.location.distance,venue.id
0,Rockview Hotel,Hotel,6.448105,3.360831,2861,4e1cbebe7d8b8fe8c30af7b0
1,Film House Cinema,Multiplex,6.490242,3.357371,4571,50ccbc9ce4b0d07024dd14f4
2,tank and tummy festac,Fast Food Restaurant,6.477172,3.332893,2477,4d4ea5e9ae4e60fce52c02a2
3,Mega Chicken Chinese Restaurant,Chinese Restaurant,6.471061,3.312721,3122,54ce24ae498e6f49a707f987
4,Apapa Shopping Mall,Shopping Mall,6.447551,3.365189,3342,5505878c498eb6579786bfd7


Let's plot the map of 'Ajeromi-LgA'

In [196]:
plot_map('Ajeromi-Ifelodun')

Next let's see the number of venues returned for the third LGA and the first 5 venues

In [197]:
# First let's confirm the name
copy_lagos_df.index[2]

'Kosofe'

In [198]:
Kosofe_lga_venues = return_venues(copy_lagos_df.index[2])

print('Total number of venues returned is {}'.format(len(Kosofe_lga_venues)))

Kosofe_lga_venues.head()

Total number of venues returned is 78


,venue.name,venue.categories,venue.location.lat,venue.location.lng,venue.location.distance,venue.id
0,De Prince Supermarket,Department Store,6.554255,3.388933,4210,4fce4c1ae4b0d4988249ad4a
1,Mende Suya Spot,BBQ Joint,6.568890,3.371953,4960,4c5af9572091a593d5b160d0
2,Domino's Pizza,Pizza Place,6.548545,3.388281,4740,552574af498e9338ad7c4479
3,Village Lounge,Bar,6.565680,3.371683,5105,4e7de58dd3e3294a67c5b155
4,Zen Garden Chinese Resturant,Chinese Restaurant,6.585504,3.357131,6393,4cbb0b289552b60c6994dd8b


Now let's plot the map of Kosofe LGA

In [199]:
plot_map('Kosofe')